# **Computing the VIX**

**Original Author: Austin Clime**

**Dated: 9 June, 2021**

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
from datetime import timedelta
import pytz
import datetime

import pandas as pd
import numpy as np

import gspread
from oauth2client.client import GoogleCredentials

from numba import jit

#! pip install tabula-py
#! pip install tabulate

#from tabula import read_pdf
#from tabulate import tabulate

# **Acknowledgments**

### I would like to thank **Mark Loewenstein** and **Steve Heston** for providing sample code, notes and logic used in this project.

# **Introduction**

### The VIX originates from trying to price the log contract, which pays off log($\frac{S(T)}{S(0)}$) at time T. The log contract gives us insight about expected future volatility.

### The value of the log contract is: $E^*[e^{-rT} log(\frac{S(T)}{S(0)})] = e^{-rT}E^*[rT - \frac{1}{2} \int_{0}^{T} \sigma(t)^2 \,dt]$

### The objective of the VIX is to replicate the payoff of the log contract using a portfolio of out-of-the money options on the S&P 500 Index.

### Let $ \hat{\sigma}^2 = \frac{1}{T} E^*  [\int_{0}^{T} \sigma(t)^2 \,dt ]$ and $F_{0,T} = e^{rT}S(0)$

### Then $ \hat{\sigma}^2 = \frac{2e^{rT}}{T} (\int_{0}^{F_{0,T}} \frac{1}{K^2} P(K) \,dK + \int_{F_{0,T}}^{\infty} \frac{1}{K^2} C(K) \,dK)$

### where P(K) is a put with strike price K and C(K) is a call strike price K

### We can think about the VIX as a portfolio of options that approximates the following: $\hat{\sigma} = \frac{2e^{rT}}{T} (\,\sum_{K_i \leq K_0}{\frac{\Delta K_i}{K_i^2}P(K_i)} +  \sum_{K_i > K_0}{\frac{\Delta K_i}{K_i^2}C(K_i)} \,)-\frac{1}{T}(\frac{e^{rT}S(0)}{K_0} - 1)^2$

### where $K_0$ is the first strike price below $e^{rT}S(0), \Delta K_i = \frac{K_{i} - K_{i-1}}{2}$. The last term is a correction term for the fact that there may not be an option with strike price equal to $e^{rT}S(0)$. It is the interval between strike prices - half the difference between the strike on either side of $K_i$.

### (**Note**: $\Delta$K for the lowest strike is simply the difference between the lowest strike and the next higher strike. Likewise, $\Delta$K for the highest strike is the difference between the highest strike and the next lower strike.)

### Q($K_i$) is the midpoint of the bid-ask spread for each option with strike $K_i$

### The framework below for pricing the VIX is adapted from the CBOE's 2009 White Paper. I will mimic the paper's computations and contents here in this notebook. The paper can be accessed here for reference: https://www.optionseducation.org/referencelibrary/white-papers/page-assets/vixwhite.aspx

# **Mechanics of the VIX**

### VIX measures 30-day expected volatility of the S&P 500 Index. The components of VIX are near- and next-term put and call options, usually in the first and second SPX contract months. “Near-term” options must have at least one week to expiration; a requirement intended to minimize pricing anomalies that might occur close to expiration. When the near-term options have less than a week to expiration, VIX “rolls” to the second and third SPX contract months. For example, on the second Friday in June, VIX would be calculated using SPX options expiring in June and July. On the following Monday, July would replace June as the “near-term” and August would replace July as the “next-term.”

### For the purpose of calculating time to expiration, SPX options are deemed to “expire” at the open of trading on SPX settlement day - the third Friday of the month.

### The VIX calculation measures time to expiration, T, in calendar days and divides each day into minutes in order to replicate the precision that is commonly used by professional option and volatility traders. The time to expiration is given by the following expression:

### T = {$M_{Current\,day} + M_{Settlement\,day} + M_{Other\,days}$} / Minutes in a year

### Where:
### $M_{Current\,day}$ = minutes remaining until midnight of the current day
### $M_{Settlement\,day}$ = minutes from midnight until 8:30 a.m. on SPX settlement day
### $M_{Other\,days}$ = total minutes in the days between current day and settlement day

In [ ]:
def second_friday(year, month):
    """Return datetime.date for monthly option expiration given year and month"""
    #second = datetime.date(year, month, 8) # The 8th is the lowest third day in the month
    second = datetime.datetime(year, month, 8, 8, 30) # The 8th is the lowest third day in the month
    w = second.weekday()                   # What day of the week is the 8th?
    if w != 4:                             # Friday is weekday 4
        second = second.replace(day=(8 + (4 - w) % 7)) # Replace just the day (of month)
    return second

In [ ]:
#pd.date_range('2019-12-31','2020-12-31',freq='WOM-2FRI')

In [ ]:
year = 2020
month = 7

near_term_exp = second_friday(year, month)
next_term_exp = second_friday(year, month + 1)

near_term_exp_ch = pytz.timezone('America/Chicago').localize(near_term_exp)
next_term_exp_ch = pytz.timezone('America/Chicago').localize(next_term_exp) #chicago timezone

td = near_term_exp_ch + timedelta(days=-9)
td

datetime.datetime(2020, 7, 1, 8, 30, tzinfo=<DstTzInfo 'America/Chicago' CDT-1 day, 19:00:00 DST>)

In [ ]:
days_per_year = 365 #calendar, not trading days
minutes_per_year = days_per_year * 24 * 60
minutes_per_day = 24 * 60
#minutes_per_year

M_current_day = datetime.datetime.combine(td.date(), datetime.datetime.max.time()) #midnight of the current day
M_current_day = pytz.timezone('America/Chicago').localize(M_current_day)
M_current_day -= td
M_current_day = round(M_current_day.seconds / 60)
#print(type(M_current_day))
#M_current_day

def M_settlement_day(settlement_date):
  M_settlement_day = datetime.datetime.combine(settlement_date.date(), datetime.datetime.min.time()) #midnight of the current day
  M_settlement_day = pytz.timezone('America/Chicago').localize(M_settlement_day)
  M_settlement_day = settlement_date - M_settlement_day
  M_settlement_day = M_settlement_day.seconds / 60
  #print(type(M_settlement_day))
  return M_settlement_day

#print(M_settlement_day(near_term_exp_ch))

def M_other_days(settlement_day, current_day):
  settlement_day = datetime.datetime.combine(settlement_day.date(), datetime.datetime.min.time())
  current_day = datetime.datetime.combine(current_day.date(), datetime.datetime.max.time())
  M_other_days = settlement_day - current_day
  M_other_days = M_other_days.days * (24*60)
  #print(type(M_other_days))
  return M_other_days

#M_other_days(settlement_day = near_term_exp_ch, current_day=td)
#M_other_days(settlement_day = td + timedelta(days=37), current_day=td)

#print(8 * 24 * 60)
#print(51840 / 60 /24)


def opt_time_to_mat(M_current_day, M_settlement_day, M_other_days):
  days_per_year = 365
  minutes_per_year = days_per_year * 24 * 60
  T = M_current_day + M_settlement_day + M_other_days
  T /= minutes_per_year
  return T

T1 = opt_time_to_mat(M_current_day=M_current_day,
                     M_settlement_day=M_settlement_day(near_term_exp_ch),
                     M_other_days=M_other_days(settlement_day = near_term_exp_ch, current_day=td))
T2 = opt_time_to_mat(M_current_day=M_current_day,
                     M_settlement_day=M_settlement_day(td + timedelta(days=37)),
                     M_other_days=M_other_days(settlement_day = td + timedelta(days=37), current_day=td))

print(f'Near-term time-to-maturity: {T1:.7f} years')
print(f'Next-term time-to-maturity: {T2:.7f} years')

#print(510/60 + 930/60)


Near-term time-to-maturity: 0.0246575 years
Next-term time-to-maturity: 0.1013699 years


### The risk-free interest rate, R, is the bond-equivalent yield of the U.S. T-bill maturing closest to the expiration dates of relevant SPX options. As such, the VIX calculation may use different risk-free interest rates for near- and next-term options. In this example, however, assume that R = 0.38% for both sets of options.

In [ ]:
R = 0.0038

# ***STEP 1*** - Select the options to be used in the VIX calculation

In [ ]:
gc = gspread.authorize(GoogleCredentials.get_application_default())
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1aURlVGpqGGHVXQb0yuU6l2h0EBZaqaZBziHGQMZ-hq0/edit#gid=397729323')
near_term_opts = wb.worksheet('Near-Term Options')
near_term_opts = pd.DataFrame( near_term_opts.get_all_records() )
near_term_opts.set_index('Strike', drop=True,inplace=True)

next_term_opts = wb.worksheet('Next-Term Options')
next_term_opts = pd.DataFrame( next_term_opts.get_all_records() )
next_term_opts.set_index('Strike', drop=True,inplace=True)

### The selected options are out-of-the-money SPX calls and out-of-the-money SPX puts centered around an at-the-money strike price, $K_0$. Only SPX options quoted with non-zero bid prices are used in the VIX calculation.

### **One important note**: as volatility rises and falls, the strike price range of options with non- zero bids tends to expand and contract. As a result, the number of options used in the VIX calculation may vary from month-to-month, day-to-day and possibly, even minute-to- minute.

### For each contract month:

### a) Determine the forward SPX level, F, by identifying the strike price at which the absolute difference between the call and put prices is smallest. The call and put prices in the following table reflect the average of each option’s bid / ask quotation. As shown below, the difference between the call and put prices is smallest at the **920** strike for both the near- and next-term options.

In [ ]:
near_term_opts['Absolute Difference'] = near_term_opts[['Call Bid', 'Call Ask']].mean(axis=1)
near_term_opts['Absolute Difference'] -= near_term_opts[['Put Bid', 'Put Ask']].mean(axis=1)
near_term_opts['Absolute Difference'] = near_term_opts['Absolute Difference'].abs()
#near_term_opts.loc[920]
min_diff_near = near_term_opts['Absolute Difference'].idxmin()

next_term_opts['Absolute Difference'] = next_term_opts[['Call Bid', 'Call Ask']].mean(axis=1)
next_term_opts['Absolute Difference'] -= next_term_opts[['Put Bid', 'Put Ask']].mean(axis=1)
next_term_opts['Absolute Difference'] = next_term_opts['Absolute Difference'].abs()
#next_term_opts.loc[920]
min_diff_next = next_term_opts['Absolute Difference'].idxmin()

### Using the 920 call and put in each contract month and the formula,
### F = Strike Price + $e^{RT}$ × (Call Price – Put Price),

#### the forward index prices, F1 and F2, for the near- and next-term options, respectively, are:

In [ ]:
row_min_diff_near = near_term_opts[ near_term_opts['Absolute Difference'] == near_term_opts['Absolute Difference'].min() ]
Call_near = row_min_diff_near[['Call Bid', 'Call Ask']].mean(axis=1).values
Put_near = row_min_diff_near[['Put Bid', 'Put Ask']].mean(axis=1).values

F1 = float(  min_diff_near + np.exp(R*T1) * (Call_near - Put_near) )
#--------------------------------------------------------------------------------------#
row_min_diff_next = next_term_opts[ next_term_opts['Absolute Difference'] == next_term_opts['Absolute Difference'].min() ]
Call_next = row_min_diff_next[['Call Bid', 'Call Ask']].mean(axis=1).values
Put_next = row_min_diff_next[['Put Bid', 'Put Ask']].mean(axis=1).values

F2 = float(  min_diff_next + np.exp(R*T2) * (Call_next - Put_next) )

### b) Next, determine **$K_0$** -- the strike price immediately below the forward index level, F -- for the near- and next-term options. In this example, **$K_{0,1}$** = 920 and **$K_{0,2}$** = 920.

### c) Select out-of-the-money put options with strike prices < $K_0$. Start with the put strike immediately lower than $K_0$ and move to successively lower strike prices. Exclude any put option that has a bid price equal to zero (i.e., no bid). As shown below, once two puts with consecutive strike prices are found to have zero bid prices, no puts with lower strikes are considered for inclusion.

### d) Next, select out-of-the-money call options with strike prices > $K_0$. Start with the call strike immediately higher than $K_0$ and move to successively higher strike prices, excluding call options that have a bid price of zero. As with the puts, once two consecutive call options are found to have zero bid prices, no calls with higher strikes are considered. (**Note that the 1250 call option is not included despite having a non- zero bid price.**)

In [ ]:
#@jit(parallel=True, cache=True)
def cons_bids(col, threshold=2, opt_type='Put'):
  #Puts
  indicator = 0
  if opt_type == 'Put':
    for i in sorted(col.index, reverse=True):
      if indicator == threshold:
        #return col.loc[i]
        #return i
        return col[ col.index > i][1:,]

      else:
        if col.loc[i] == 0:
          indicator += 1
        #else:

  #Calls
  else:
    for i in sorted(col.index, reverse=False):
      if indicator == threshold:
        return col[ col.index < i][:-1,]

      else:
        if col.loc[i] == 0:
          indicator += 1

In [ ]:
#This works, but I fear it is not general enough - should be re-written - next_puts_used is an example of loss of generality
#Use this one
def cons_bids(col, threshold=2, opt_type='Put'):
  #Puts
  indicator = 0
  if opt_type == 'Put':
    for i in sorted(col.index, reverse=True):
      if indicator == threshold:
        #return col.loc[i]
        #return i
        return col[ col.index > i][1:,]

      else:
        if col.loc[i] == 0:
          indicator += 1
        #else:
    if indicator < threshold:
      print('zero ind')
      return col

  #Calls
  else:
    for i in sorted(col.index, reverse=False):
      if indicator == threshold:
        return col[ col.index < i][:-1,]

      else:
        if col.loc[i] == 0:
          indicator += 1

In [ ]:


K0_near = near_term_opts.index[near_term_opts.index < F1 ][-1]
K0_next = next_term_opts.index[next_term_opts.index < F2 ][-1]

#Near-Term Options
#Puts
near_puts_used = near_term_opts[ near_term_opts.index < K0_near]
near_puts_used = cons_bids(col=near_puts_used['Put Bid'], threshold=1, opt_type='Put')
#Calls
near_calls_used = near_term_opts[ near_term_opts.index > K0_near]
near_calls_used = cons_bids(col=near_calls_used['Call Bid'], threshold=1, opt_type='Call')
near_puts_used


#Next-Term Options
#Puts
next_puts_used = next_term_opts[ next_term_opts.index < K0_next]
next_puts_used = cons_bids(col=next_puts_used['Put Bid'], threshold=2, opt_type='Put')
#Calls
next_calls_used = next_term_opts[ next_term_opts.index > K0_next]
next_calls_used = cons_bids(col=next_calls_used['Call Bid'], threshold=1, opt_type='Call')

zero ind


In [ ]:
#sorted( near_puts_used['Put Bid'].index, reverse=True )

### e) Finally, select both the put and call with strike price **$K_0$**. Notice that two options are selected at **$K_0$**, while a single option, either a put or a call, is used for every other strike price.)

### The following table contains the options used to calculate the VIX in this example. VIX uses the average of quoted bid and ask, or mid-quote, prices for each option selected. The K0 put and call prices are averaged to produce a single value. The price used for the 920 strike in the near-term is, therefore, (37.15 + 36.65)/2 = 36.90; and the price used in the next-term is (61.55 + 60.55)/2 = 61.05.

In [ ]:
#min_diff_near
#min_diff_next

920

In [ ]:
#near_term_opts.loc[min_diff_near][['Call Bid', 'Call Ask', 'Put Bid', 'Put Ask']].mean()

float

In [ ]:
near_opts_used = near_term_opts.loc[near_calls_used.index.append(near_puts_used.index)]
near_opts_used['Option Type'] = near_opts_used.index
near_opts_used['Option Type'] = near_opts_used['Option Type'].apply( lambda x: 'Call' if x > min_diff_near else 'Put')
near_opts_used['Mid-quote Price'] = 0

#near_opts_used['Mid-quote Price'] =  near_opts_used['Mid-quote Price'].apply( lambda x: x[['Call Bid', 'Call Ask']].mean(axis=1).values if x['Option Type'] == 'Call' else x[['Put Bid', 'Put Ask']].mean(axis=1).values)
near_opts_used['Call Midquote'] = near_opts_used[['Call Bid', 'Call Ask']].mean(axis=1).values
near_opts_used['Put Midquote'] = near_opts_used[['Put Bid', 'Put Ask']].mean(axis=1).values

near_opts_used['Mid-quote Price'] =  near_opts_used.apply(lambda x : x['Call Midquote'] if x['Option Type'] == 'Call' else x['Put Midquote'], axis=1)
near_opts_used = near_opts_used[['Mid-quote Price', 'Option Type']]

put_call_avg_near = near_term_opts.loc[min_diff_near][['Call Bid', 'Call Ask', 'Put Bid', 'Put Ask']].mean()
new_row = pd.Series(data={'Mid-quote Price':put_call_avg_near, 'Option Type':'Put/Call Average'}, name=min_diff_near)
near_opts_used = near_opts_used.append(new_row, ignore_index=False)

near_opts_used.sort_index(inplace=True)
near_opts_used

In [ ]:
next_opts_used = next_term_opts.loc[next_calls_used.index.append(next_puts_used.index)]
next_opts_used['Option Type'] = next_opts_used.index
next_opts_used['Option Type'] = next_opts_used['Option Type'].apply( lambda x: 'Call' if x > min_diff_next else 'Put')
next_opts_used['Mid-quote Price'] = 0

next_opts_used['Call Midquote'] = next_opts_used[['Call Bid', 'Call Ask']].mean(axis=1).values
next_opts_used['Put Midquote'] = next_opts_used[['Put Bid', 'Put Ask']].mean(axis=1).values

next_opts_used['Mid-quote Price'] =  next_opts_used.apply(lambda x : x['Call Midquote'] if x['Option Type'] == 'Call' else x['Put Midquote'], axis=1)
next_opts_used = next_opts_used[['Mid-quote Price', 'Option Type']]

put_call_avg_next = next_term_opts.loc[min_diff_next][['Call Bid', 'Call Ask', 'Put Bid', 'Put Ask']].mean()
new_row = pd.Series(data={'Mid-quote Price':put_call_avg_next, 'Option Type':'Put/Call Average'}, name=min_diff_next)
next_opts_used = next_opts_used.append(new_row, ignore_index=False)

next_opts_used.sort_index(inplace=True)
next_opts_used

# ***STEP 2*** - Calculate volatility for both near-term and next-term options

### Applying the VIX formula (1) to the near-term and next-term options with time to expiration of $T_1$ and $T_2$, respectively, yields:

### $\sigma^2_1 = \frac{2}{T_1} \sum_i \frac{\Delta K_i}{K_i^2}e^{RT_i} Q(K_i) - \frac{1}{T_1}[\frac{F_1}{K_0}-1]^2$

### $\sigma^2_2 = \frac{2}{T_2} \sum_i \frac{\Delta K_i}{K_i^2}e^{RT_2} Q(K_i) - \frac{1}{T_2}[\frac{F_2}{K_0}-1]^2$

### VIX is an amalgam of the information reflected in the prices of all of the selected options. The contribution of a single option to the VIX value is proportional to ∆K and the price of that option, and inversely proportional to the square of the option’s strike price.

### Generally, ∆$K_i$ is half the difference between the strike prices on either side of $K_i$. For example, the ∆K for the next-term 300 Put is 75: ∆K300 Put = (350 – 200)/2. At the upper and lower edges of any given strip of options, ∆$K_i$ is simply the difference between $K_i$ and the adjacent strike price. In this example, the 400 Put is the lowest strike in the strip of near-term options and 425 is the adjacent strike. Therefore, ∆$K_{400 Put}$ = 25 (i.e., 425 – 400).

### The contribution of the near-term 400 Put is given by:

### $\frac{\Delta K_{400 Put}}{K^2_{400 Put}} e^{RT_1} Q(400 Put) = \frac{25}{400} e^{(.0038)0.0246575} (0.125) = 0.0000195$


### A similar calculation is performed for each option. The resulting values for the near-term options are then summed and multiplied by 2/$T_1$ . Likewise, the resulting values for the next-term options are summed and multiplied by 2/$T_2$ . The table below summarizes the results for each strip of options.

In [ ]:
#next_opts_used

In [ ]:
next_opts_used['Contribution by Strike'] = next_opts_used.index
next_opts_used['Contribution by Strike'] = next_opts_used['Contribution by Strike'].shift(-1) - next_opts_used['Contribution by Strike'].shift(1)
next_opts_used['Contribution by Strike'] /= 2
idx_num = next_opts_used.columns.tolist().index('Contribution by Strike')
#next_opts_used.iloc[0]['Contribution by Strike'] = (next_opts_used.iloc[1].name - next_opts_used.iloc[0].name) / 2
next_opts_used.iloc[0, idx_num] = (next_opts_used.iloc[1].name - next_opts_used.iloc[0].name)
next_opts_used.iloc[-1,idx_num] = (next_opts_used.iloc[-1].name - next_opts_used.iloc[-2].name)
next_opts_used['Contribution by Strike'] /= next_opts_used.index**2
next_opts_used['Contribution by Strike'] *= np.exp(R*T2)
next_opts_used['Contribution by Strike'] *= next_opts_used['Mid-quote Price']
next_opts_used

In [ ]:
near_opts_used['Contribution by Strike'] = near_opts_used.index
near_opts_used['Contribution by Strike'] = near_opts_used['Contribution by Strike'].shift(-1) - near_opts_used['Contribution by Strike'].shift(1)
near_opts_used['Contribution by Strike'] /= 2
idx_num = near_opts_used.columns.tolist().index('Contribution by Strike')
near_opts_used.iloc[0, idx_num] = (near_opts_used.iloc[1].name - near_opts_used.iloc[0].name)
near_opts_used.iloc[-1,idx_num] = (near_opts_used.iloc[-1].name - near_opts_used.iloc[-2].name)
near_opts_used['Contribution by Strike'] /= near_opts_used.index**2
near_opts_used['Contribution by Strike'] *= np.exp(R*T1)
near_opts_used['Contribution by Strike'] *= near_opts_used['Mid-quote Price']
near_opts_used

### Next, calculate $\frac{1}{T} [\frac{F}{K_0} - 1]^2$ for the near-term ($T_1$) and the next-term ($T_2$):

### $\frac{1}{T_1} [\frac{F_1}{K_0} - 1]^2 = \frac{1}{0.0246575} [\frac{920.50005}{920} - 1]^2$ = 0.0000120

### $\frac{1}{T_2} [\frac{F_2}{K_0} - 1]^2 = \frac{1}{0.1013699} [\frac{921.00039}{920} - 1]^2$ = 0.0000117

### Now calculate $\sigma_1^2$ and $\sigma_2^2$:

### $\sigma^2_1 = \frac{2}{T_1} \sum_i \frac{\Delta K_i}{K_i^2}e^{RT_i} Q(K_i) - \frac{1}{T_1}[\frac{F_1}{K_0}-1]^2$ = 0.4727799 - 0.0000120 = **0.4727679**

### $\sigma^2_2 = \frac{2}{T_2} \sum_i \frac{\Delta K_i}{K_i^2}e^{RT_2} Q(K_i) - \frac{1}{T_2}[\frac{F_2}{K_0}-1]^2$ = 0.3668297 - 0.0000117 = **0.3668180**

In [ ]:
first_term_near = near_opts_used['Contribution by Strike'].sum()
first_term_near *= (2/T1)
second_term_near = (F1/K0_near)
second_term_near -= 1
second_term_near **= 2
second_term_near *= (1/T1)
sigma_square_near = first_term_near - second_term_near
print(sigma_square_near)

first_term_next = next_opts_used['Contribution by Strike'].sum()
first_term_next *= (2/T2)
second_term_next = (F2/K0_next)
second_term_next -= 1
second_term_next **= 2
second_term_next *= (1/T2)
sigma_square_next = first_term_next - second_term_next
print(sigma_square_next)

In [ ]:
#next_opts_used.['Contribution by Strike'].sum() * (2/T2)

### ***STEP 3*** - Calculate the 30-day weighted average of $\sigma^2_1$ and $\sigma^2_2$. Then take the square root of that value and multiply by 100 to get VIX

### VIX = 100 x $\sqrt{\{T_1\sigma_1^2 [\frac{N_{T_2} - N_{T_{30}} }{N_{T_2} - N_{T_1}}] + T_2\sigma_2^2 [\frac{N_{T_{30}} - N_{T_1} }{N_{T_2} - N_{T_1}}] \}x\frac{N_{365}}{N_{30}}}$

### When the near-term options have less than 30 days to expiration and the next-term options have more than 30 days to expiration, the resulting VIX value reflects an interpolation of $\sigma_1^2$ and $\sigma_2^2$ ;i.e., each individual weight is less than or equal to 1 and the sum of the weights equals 1.
\
### At the time of the VIX “roll,” both the near-term and next-term options have more than 30 days to expiration. The same formula is used to calculate the 30-day weighted average, but the result is an extrapolation of $\sigma_1^2$ and $\sigma_2^2$ ;i.e., the sum of the weights is still 1, but the near-term weight is greater than 1 and the next-term weight is negative (e.g., 1.25 and – 0.25).

### $N_{T1}$ = number of minutes to settlement of the near-term options (12,960)
### $N_{T2}$ = number of minutes to settlement of the next-term options (53,280)
### $N_{T30}$ = number of minutes in 30 days (30 × 1,440 = 43,200)
### $N_{T365}$ = number of minutes in a 365-day year (365 ×1,440 = 525,600)

In [ ]:
#minutes_per_year
minsutes_30_days = 60 * 24 * 30
N_T1 = T1 * 365 * 24 * 60
N_T2 = T2 * 365 * 24 * 60

vix = T1*sigma_square_near * ( (N_T2 - minsutes_30_days) / (N_T2-N_T1) )
vix += T2*sigma_square_next * ( (minsutes_30_days-N_T1) / (N_T2-N_T1) )
vix *= (minutes_per_year/minsutes_30_days)
vix = np.sqrt(vix)
vix *= 100
vix

61.19559541023823

### VIX = 100 x $\sqrt{\{0.0246575 x 0.4727679 x [\frac{53,280 - 43,200}{53,280 - 12,960}] + 0.1013699 x 0.3668180 x [\frac{43,200 - 12,960}{53,280 - 12,960}] \}x\frac{525,600}{43,200} }$

### **VIX = 100 x 0.612179986 = 61.22**

# **Scrap Work**

In [ ]:
! pip install tabula-py
import tabula
file = '/content/drive/My Drive/personal/VIX/vixwhite.pdf'
tables = tabula.read_pdf(file, pages = "all", multiple_tables = True)

#df = read_pdf(file,pages="all")
#print(tabulate(df))

Got stderr: Jun 12, 2021 3:13:46 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode



# **Sources**

https://www.optionseducation.org/referencelibrary/white-papers/page-assets/vixwhite.aspx

Historical prices for VIX, VXO and CBOE’s other volatility indexes may be found on the CBOE website at http://www.cboe.com/micro/IndexSites.aspx under CBOE Volatility Indexes.

https://ispycode.com/Blog/python/2016-07/Get-Midnight-Today

https://stackoverflow.com/questions/28680896/how-can-i-get-the-3rd-friday-of-a-month-in-python/28681097

https://stackoverflow.com/questions/31304890/wrong-aware-datetime-with-pytz-and-america-chicago

https://github.com/skyfielders/python-skyfield/issues/409

https://www.kite.com/python/answers/how-to-convert-a-timedelta-to-days,-hours,-and-minutes-in-python

http://theautomatic.net/2019/05/24/3-ways-to-scrape-tables-from-pdfs-with-python/

https://medium.com/analytics-vidhya/colab-and-google-sheets-surprisingly-powerful-combination-for-data-science-part-1-bbbb11cbd8e

https://simplypdf.com/Excel

https://docs.gspread.org/en/latest/user-guide.html#getting-all-values-from-a-worksheet-as-a-list-of-lists

https://www.overleaf.com/learn/latex/List_of_Greek_letters_and_math_symbols

https://stackoverflow.com/questions/61298766/idmin-and-idmax-in-a-series-not-working

https://www.geeksforgeeks.org/select-row-with-maximum-and-minimum-value-in-pandas-dataframe/

https://stackoverflow.com/questions/44577622/lambda-data-frame-reference-a-value-in-another-column

https://pythonexamples.org/pandas-dataframe-add-append-row/

https://pythonexamples.org/python-find-index-of-item-in-list/

https://stackoverflow.com/questions/29241836/select-multiple-columns-by-labels-in-pandas

https://stackoverflow.com/questions/60334671/pandas-dataframe-how-to-find-consecutive-rows-that-meet-some-conditions

https://datascience.stackexchange.com/questions/20587/find-the-consecutive-zeros-in-a-dataframe-and-do-a-conditional-replacement

https://realpython.com/pandas-sort-python/

# **Drive Links**

drive: https://drive.google.com/drive/u/0/folders/14bOw2BBUUBya_Onrq6CE6u5Hip2GljIJ

spreadsheet: https://docs.google.com/spreadsheets/d/1aURlVGpqGGHVXQb0yuU6l2h0EBZaqaZBziHGQMZ-hq0/edit#gid=1010367841